# 要約 
このJupyter Notebookは、Llama-3 8bモデルをTPU(Super TPU)でトレーニングすることに焦点を当てています。特に、大規模言語モデル(LLM)のトレーニング過程を詳細に説明し、実際にトレーニングを行うための実装方法を示しています。

### 取り組んでいる問題
ノートブックは、Llama-3のLPF（Low-Rank Adaptation、LoRA）と呼ばれる技術を使用して、TPU上でのモデルのトレーニングを効率化する問題に取り組んでいます。具体的には、チャットボットの応答に対するユーザーの好みを予測するためのモデルをトレーニングし、ユーザインタラクションの改善を目指しています。

### 使用している手法とライブラリ
- **ライブラリ**：
  - `transformers`: Hugging Faceのライブラリを用いて、LLMのモデルやトークナイザーを扱っています。
  - `torch`: PyTorchを使用して、モデルのトレーニングとテストを行います。
  - `torch_xla`: TPUでの計算をサポートするために、XLA（Accelerated Linear Algebra）関連の機能を利用しています。
  - `peft`: 特にLoRAに関連するパラメータの効率的な調整を行います。

- **手法**:
  - **LoRA (Low-Rank Adaptation)**: 低ランク行列の近似を使用して、モデルのトレーニングを効率的に行い、パラメータの数を削減します。
  - **クロスエントロピー損失関数**: モデルの損失を評価するために使用されます。
  - **コサイン学習率スケジュール**: 学習率を調整しつつトレーニングを進めるために用いられます。
  
ノートブックは、トレーニングデータを読み込み、データ前処理を行い、トークナイザーを使ってデータをトークナイズします。その後、モデルを定義し、TPU環境におけるデータのシャーディングとパラメータの設定を行います。最終的に、トレーニングループ内でモデルを訓練し、損失の履歴をプロットして視覚的に訓練状況を確認します。

### 結論
全体として、このノートブックはLlama-3モデルをTPUでトレーニングするための具体的な手順と実装を示し、さらなるデータやハイパーパラメータの調整によるトレーニングの最適化の可能性についても言及しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する初心者がつまずきそうな専門用語の簡単な解説を示します。

1. **TPU (Tensor Processing Unit)**:
   - Googleが開発した専用のハードウェアで、特に機械学習のトレーニングと推論を効率的に行うために設計されています。CPUやGPUよりも高速にディープラーニングモデルを処理することができます。

2. **BFloat16**:
   - GoogleのTPUで使用される数値表現の一つで、16ビットの浮動小数点数です。BFloat16は、効率的な計算を可能にしつつ、深層学習の精度を維持するために使用されます。

3. **LoRA (Low-Rank Adaptation)**:
   - 事前トレーニングされたモデルに対して、パラメータの数を抑えて微調整を行う手法です。基本的なモデルに対して追加の低ランクの行列を学習させることで、リソースを節約しつつ高い性能を実現します。

4. **SPMD (Single Program Multiple Data)**:
   - 同じプログラムが複数のデータに対して並行して実行されるプログラミングモデルです。TPUやGPUの使用において、データパラレルな計算を行う際によく用いられます。

5. **シャーディング (Sharding)**:
   - 大規模データセットやモデルを複数のデバイスに分散（シャード）して処理する技術です。これにより、計算負荷を軽減し、効率的な学習が可能になります。

6. **メッシュ (Mesh)**:
   - データプレシジョン（precision）や並列処理のために、TPUやGPUデバイスの構造を定義することに関連する概念です。通常、デバイスの形状や分割方法を決定します。

7. **クロスエントロピー損失 (Cross Entropy Loss)**:
   - 主に分類問題で使用される損失関数で、予測した確率と実際のラベルの分布間の差異を評価します。出力確率がどれだけ真のラベルに近いかを測ります。

8. **アテンションマスク (Attention Mask)**:
   - Transformerモデルにおいて、どのトークンが注意の対象であるかを示すためのマスクです。無関係なトークンを考慮しないようにするために使用されます。

9. **ウィアモード (Warmup Steps)**:
   - 学習率を徐々に増加させるための初期ステップです。急激な学習率の変化を防ぐことで、モデルの収束を安定させる役割があります。

10. **トークナイザー (Tokenizer)**:
   - テキストをモデルが処理しやすい形式（トークン）に変換するコンポーネントです。トークン化は、自然言語処理の初期ステップとして重要です。

これらの用語の理解は、このJupyter Notebookの内容を深く理解し、トレーニングプロセスを効果的に進めるために役立つでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#  Llama-3 8b [TPU Train]

Learning to train llms on tpu, Hope this will help you too!

Notebook inspired from:

* [LLM detect AI comp Mistral-7B](https://www.kaggle.com/code/hotchpotch/train-llm-detect-ai-comp-mistral-7b/notebook)
* [DAIGT Mistral-7B TPU BFloat16 [Train]](https://www.kaggle.com/code/markwijkhuizen/daigt-mistral-7b-tpu-bfloat16-train)
* [LLAMA 2 13B on TPU (Training)](https://www.kaggle.com/code/defdet/llama-2-13b-on-tpu-training)


Prerequisite: Access to using llama-3

Note: This is only training notebook, you can find inference notebook [here](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b)

Please upvote if you learn or find this helpful!

# Import libs 

</div>
<div class="column-right">

# 日本語訳

# Llama-3 8b [TPUトレーニング]

TPU上でLLMをトレーニングする方法を学ぶことができます。これがあなたにも役立つことを願っています！

ノートブックは以下からインスパイアを受けています：

* [LLM検出AIコンペ Mistral-7B](https://www.kaggle.com/code/hotchpotch/train-llm-detect-ai-comp-mistral-7b/notebook)
* [DAIGT Mistral-7B TPU BFloat16 [トレーニング]](https://www.kaggle.com/code/markwijkhuizen/daigt-mistral-7b-tpu-bfloat16-train)
* [LLAMA 2 13B on TPU (トレーニング)](https://www.kaggle.com/code/defdet/llama-2-13b-on-tpu-training)

前提条件：Llama-3の使用権限

注意：これはトレーニング専用のノートブックです、推論ノートブックは[こちら](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b)から見つけることができます。

学びや役立つことがあれば、投票していただけると嬉しいです！

# ライブラリのインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Install libs
!pip install -qq peft==0.6.0
!pip install -qq bitsandbytes==0.41.1
!pip install -qq accelerate==0.24.1
!pip install -qq transformers==4.35.0
!pip install -qq torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q 
!pip install -qq torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q
!pip uninstall -qq tensorflow -y # If we don't do this, TF will take over TPU and cause permission error for PT
!cp /kaggle/input/utils-xla/spmd_util.py . # From this repo: https://github.com/HeegyuKim/torch-xla-SPMD
```

</div>
<div class="column-right">

# 日本語訳

```python
# ライブラリをインストール
!pip install -qq peft==0.6.0  # PEFTライブラリのインストール
!pip install -qq bitsandbytes==0.41.1  # BitsAndBytesライブラリのインストール
!pip install -qq accelerate==0.24.1  # Accelerateライブラリのインストール
!pip install -qq transformers==4.35.0  # Transformersライブラリのインストール
!pip install -qq torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q  # CPU用のPyTorchのインストール
!pip install -qq torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q  # TPU用のTorch XLAのインストール
!pip uninstall -qq tensorflow -y # テンソルフローをアンインストールします。これを行わないと、TFがTPUを占有し、PyTorchに対する権限エラーが発生する可能性があります。
!cp /kaggle/input/utils-xla/spmd_util.py . # このリポジトリからファイルをコピーします: https://github.com/HeegyuKim/torch-xla-SPMD
```

</div>
</details>

In [ ]:
# ライブラリをインストール
!pip install -qq peft==0.6.0  # PEFTライブラリのインストール
!pip install -qq bitsandbytes==0.41.1  # BitsAndBytesライブラリのインストール
!pip install -qq accelerate==0.24.1  # Accelerateライブラリのインストール
!pip install -qq transformers==4.35.0  # Transformersライブラリのインストール
!pip install -qq torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q  # CPU用のPyTorchのインストール
!pip install -qq torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q  # TPU用のTorch XLAのインストール
!pip uninstall -qq tensorflow -y # テンソルフローをアンインストールします。これを行わないと、TFがTPUを占有し、PyTorchに対する権限エラーが発生する可能性があります。
!cp /kaggle/input/utils-xla/spmd_util.py . # このリポジトリからファイルをコピーします: https://github.com/HeegyuKim/torch-xla-SPMD

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import gc
import re
from time import time
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import transformers
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch.nn.functional as F

import torch_xla.debug.profiler as xp
import torch_xla.core.xla_model as xm
import torch_xla.experimental.xla_sharding as xs
import torch_xla.runtime as xr

xr.use_spmd()

from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh
from spmd_util import partition_module

tqdm.pandas()

print(f'Torch Version: {torch.__version__}')
```

</div>
<div class="column-right">

# 日本語訳

```python
import os  # osモジュールをインポート（オペレーティングシステムに関連する機能を提供）
import gc  # gcモジュールをインポート（ガーベジコレクション、不要なメモリの解放を行う）
import re  # reモジュールをインポート（正規表現を扱うための機能を提供）
from time import time  # timeモジュールからtime関数をインポート（時間計測を行うため）
import random  # randomモジュールをインポート（ランダムな数の生成を行う）
import warnings  # warningsモジュールをインポート（警告メッセージを表示するため）
import numpy as np  # NumPyライブラリをnpというエイリアスでインポート（数値計算を行うため）
import pandas as pd  # pandasライブラリをpdというエイリアスでインポート（データ操作と分析のためのライブラリ）
import matplotlib.pyplot as plt  # Matplotlibライブラリをpltというエイリアスでインポート（データの可視化を行うため）
from tqdm.auto import tqdm  # tqdmライブラリから自動的にプログレスバーをインポート

import torch  # PyTorchライブラリをインポート（機械学習のためのライブラリ）
import transformers  # Hugging FaceのTransformersライブラリをインポート（NLPモデルを使用するため）
from sklearn.metrics import accuracy_score  # scikit-learnから精度計算のための関数をインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification  # 自動トークナイザーとモデルをインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # PEFT関連のモジュールをインポート
import torch.nn.functional as F  # PyTorchのニューラルネットワークの関数をFというエイリアスでインポート

import torch_xla.debug.profiler as xp  # XLAのプロファイラをインポート（パフォーマンスの分析に使用）
import torch_xla.core.xla_model as xm  # XLAモジュールをインポート（TPUのための処理をサポート）
import torch_xla.experimental.xla_sharding as xs  # XLAのシャーディング機能をインポート（データ分散のため）
import torch_xla.runtime as xr  # XLAの実行時機能をインポート

xr.use_spmd()  # SPMD（Single Program Multiple Data）を使用する設定

from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor  # XLAシャーディングテンソルをインポート（シャーディングされたテンソルを扱う）
from torch_xla.experimental.xla_sharding import Mesh  # メッシュ構造をインポート（データ分散のため）
from spmd_util import partition_module  # モジュールをパーティションするための関数をインポート

tqdm.pandas()  # pandasの進捗バーを設定

print(f'Torch Version: {torch.__version__}')  # 現在のPyTorchのバージョンを出力
```

</div>
</details>

In [ ]:
import os  # osモジュールをインポート（オペレーティングシステムに関連する機能を提供）
import gc  # gcモジュールをインポート（ガーベジコレクション、不要なメモリの解放を行う）
import re  # reモジュールをインポート（正規表現を扱うための機能を提供）
from time import time  # timeモジュールからtime関数をインポート（時間計測を行うため）
import random  # randomモジュールをインポート（ランダムな数の生成を行う）
import warnings  # warningsモジュールをインポート（警告メッセージを表示するため）
import numpy as np  # NumPyライブラリをnpというエイリアスでインポート（数値計算を行うため）
import pandas as pd  # pandasライブラリをpdというエイリアスでインポート（データ操作と分析のためのライブラリ）
import matplotlib.pyplot as plt  # Matplotlibライブラリをpltというエイリアスでインポート（データの可視化を行うため）
from tqdm.auto import tqdm  # tqdmライブラリから自動的にプログレスバーをインポート

import torch  # PyTorchライブラリをインポート（機械学習のためのライブラリ）
import transformers  # Hugging FaceのTransformersライブラリをインポート（NLPモデルを使用するため）
from sklearn.metrics import accuracy_score  # scikit-learnから精度計算のための関数をインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification  # 自動トークナイザーとモデルをインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType  # PEFT関連のモジュールをインポート
import torch.nn.functional as F  # PyTorchのニューラルネットワークの関数をFというエイリアスでインポート

import torch_xla.debug.profiler as xp  # XLAのプロファイラをインポート（パフォーマンスの分析に使用）
import torch_xla.core.xla_model as xm  # XLAモジュールをインポート（TPUのための処理をサポート）
import torch_xla.experimental.xla_sharding as xs  # XLAのシャーディング機能をインポート（データ分散のため）
import torch_xla.runtime as xr  # XLAの実行時機能をインポート

xr.use_spmd()  # SPMD（Single Program Multiple Data）を使用する設定

from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor  # XLAシャーディングテンソルをインポート（シャーディングされたテンソルを扱う）
from torch_xla.experimental.xla_sharding import Mesh  # メッシュ構造をインポート（データ分散のため）
from spmd_util import partition_module  # モジュールをパーティションするための関数をインポート

tqdm.pandas()  # pandasの進捗バーを設定

print(f'Torch Version: {torch.__version__}')  # 現在のPyTorchのバージョンを出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Configs

</div>
<div class="column-right">

# 日本語訳

# 設定

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CFG:
    NUM_EPOCHS = 1
    BATCH_SIZE = 16
    DROPOUT = 0.05 
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    SEED = 2024 
    MAX_LENGTH = 1024 
    NUM_WARMUP_STEPS = 128
    LR_MAX = 5e-5 
    NUM_LABELS = 3 
    LORA_RANK = 4
    LORA_ALPHA = 8
    LORA_MODULES = ['o_proj', 'v_proj']
    
DEVICE = xm.xla_device() # Initialize TPU Device
```

</div>
<div class="column-right">

# 日本語訳

```python
class CFG:  # 設定を管理するクラス
    NUM_EPOCHS = 1  # エポック数（モデルをトレーニングする回数）
    BATCH_SIZE = 16  # バッチサイズ（一度に処理するデータの数）
    DROPOUT = 0.05  # ドロップアウト率（過学習を防ぐための手法）
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # 使用するモデルのパス
    SEED = 2024  # 再現性を確保するための乱数の種
    MAX_LENGTH = 1024  # 最大シーケンス長（モデルに入力する最大の長さ）
    NUM_WARMUP_STEPS = 128  # ウォームアップステップの数（学習率を調整するための初期ステップ）
    LR_MAX = 5e-5  # 最大学習率（モデルをトレーニングする際の最大の学習率）
    NUM_LABELS = 3  # ラベルの数（分類するクラスの数）
    LORA_RANK = 4  # LoRAのランク（低ランク近似のためのパラメーター）
    LORA_ALPHA = 8  # LoRAのアルファ値（スケーリングファクター）
    LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモジュールのリスト
    
DEVICE = xm.xla_device()  # TPUデバイスを初期化する
```

</div>
</details>

In [ ]:
class CFG:  # 設定を管理するクラス
    NUM_EPOCHS = 1  # エポック数（モデルをトレーニングする回数）
    BATCH_SIZE = 16  # バッチサイズ（一度に処理するデータの数）
    DROPOUT = 0.05  # ドロップアウト率（過学習を防ぐための手法）
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # 使用するモデルのパス
    SEED = 2024  # 再現性を確保するための乱数の種
    MAX_LENGTH = 1024  # 最大シーケンス長（モデルに入力する最大の長さ）
    NUM_WARMUP_STEPS = 128  # ウォームアップステップの数（学習率を調整するための初期ステップ）
    LR_MAX = 5e-5  # 最大学習率（モデルをトレーニングする際の最大の学習率）
    NUM_LABELS = 3  # ラベルの数（分類するクラスの数）
    LORA_RANK = 4  # LoRAのランク（低ランク近似のためのパラメーター）
    LORA_ALPHA = 8  # LoRAのアルファ値（スケーリングファクター）
    LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモジュールのリスト
    
DEVICE = xm.xla_device()  # TPUデバイスを初期化する

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def set_seeds(seed):
    """Set seeds for reproducibility """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
    # Set seed for all TPU cores
    xm.set_rng_state(seed, device=xm.xla_device())  

set_seeds(seed=CFG.SEED)
```

</div>
<div class="column-right">

# 日本語訳

```python
def set_seeds(seed):  # 再現性を確保するための乱数の種を設定する関数
    """再現性を確保するために種を設定します"""
    os.environ['PYTHONHASHSEED'] = str(seed)  # Pythonのハッシュシードを設定
    random.seed(seed)  # randomモジュールの種を設定
    np.random.seed(seed)  # NumPyの乱数生成器の種を設定
    torch.manual_seed(seed)  # PyTorchの手動シードを設定
    if torch.cuda.is_available():  # CUDAが利用可能な場合
        torch.cuda.manual_seed(seed)  # GPUのシードを設定
        torch.cuda.manual_seed_all(seed)  # 全てのGPUのシードを設定
        
    # すべてのTPUコアに対してシードを設定
    xm.set_rng_state(seed, device=xm.xla_device())  

set_seeds(seed=CFG.SEED)  # 定義した関数を呼び出し、設定したシードを使用して乱数の種を設定
```

</div>
</details>

In [ ]:
def set_seeds(seed):  # 再現性を確保するための乱数の種を設定する関数
    """再現性を確保するために種を設定します"""
    os.environ['PYTHONHASHSEED'] = str(seed)  # Pythonのハッシュシードを設定
    random.seed(seed)  # randomモジュールの種を設定
    np.random.seed(seed)  # NumPyの乱数生成器の種を設定
    torch.manual_seed(seed)  # PyTorchの手動シードを設定
    if torch.cuda.is_available():  # CUDAが利用可能な場合
        torch.cuda.manual_seed(seed)  # GPUのシードを設定
        torch.cuda.manual_seed_all(seed)  # 全てのGPUのシードを設定
        
    # すべてのTPUコアに対してシードを設定
    xm.set_rng_state(seed, device=xm.xla_device())  

set_seeds(seed=CFG.SEED)  # 定義した関数を呼び出し、設定したシードを使用して乱数の種を設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Tokenizer

</div>
<div class="column-right">

# 日本語訳

# トークナイザー

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

# save tokenizer to load offline during inference
tokenizer.save_pretrained('tokenizer')
```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME)  # 設定したモデル名から事前トレーニングされたトークナイザーをロード
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークン（eos_token）に設定
tokenizer.padding_side = 'right'  # パディングを右側に追加
tokenizer.add_eos_token = True  # 終了トークンを追加する設定

# 推論時にオフラインでロードできるようにトークナイザーを保存
tokenizer.save_pretrained('tokenizer')  # トークナイザーを指定したディレクトリに保存
```

</div>
</details>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME)  # 設定したモデル名から事前トレーニングされたトークナイザーをロード
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを終了トークン（eos_token）に設定
tokenizer.padding_side = 'right'  # パディングを右側に追加
tokenizer.add_eos_token = True  # 終了トークンを追加する設定

# 推論時にオフラインでロードできるようにトークナイザーを保存
tokenizer.save_pretrained('tokenizer')  # トークナイザーを指定したディレクトリに保存

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Utility function giving token length
def get_token_lengths(texts):
    # tokenize and receive input_ids for reach text
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # return length of inputs_ids for each text
    return [len(t) for t in input_ids]
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストのトークン長を取得するユーティリティ関数
def get_token_lengths(texts):  # テキストのリストを引数に取る関数
    # テキストをトークナイズし、各テキストに対するinput_idsを取得
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']  
    # 各テキストのinput_idsの長さを返す
    return [len(t) for t in input_ids]  # 各トークンの長さをリストとして返す
```

</div>
</details>

In [ ]:
# テキストのトークン長を取得するユーティリティ関数
def get_token_lengths(texts):  # テキストのリストを引数に取る関数
    # テキストをトークナイズし、各テキストに対するinput_idsを取得
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']  
    # 各テキストのinput_idsの長さを返す
    return [len(t) for t in input_ids]  # 各トークンの長さをリストとして返す

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Prepare train


</div>
<div class="column-right">

# 日本語訳

# トレーニングの準備

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

# Drop 'Null' for training
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)

print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))
```

</div>
<div class="column-right">

# 日本語訳

```python
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータをCSVファイルから読み込む

def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文をトリミングし、リストに分割
    return ' '.join(sentences)  # 文をスペースで結合して返す

# 各列に対してprocess関数を適用
train.loc[:, 'prompt'] = train['prompt'].apply(process)  # 'prompt'列を処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # 'response_a'列を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # 'response_b'列を処理

# トレーニングのために'Null'をドロップ
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 'response_a'と'response_b'が両方'null'の行のインデックスを取得
train.drop(indexes, inplace=True)  # 取得したインデックスの行を削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット

print(f"合計 {len(indexes)} 行のNullレスポンスが削除されました")
print('トレーニングサンプルの合計: ', len(train))  # トレーニングサンプルの数を出力
```

</div>
</details>

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータをCSVファイルから読み込む

def process(input_str):  # 入力文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文をトリミングし、リストに分割
    return ' '.join(sentences)  # 文をスペースで結合して返す

# 各列に対してprocess関数を適用
train.loc[:, 'prompt'] = train['prompt'].apply(process)  # 'prompt'列を処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # 'response_a'列を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # 'response_b'列を処理

# トレーニングのために'Null'をドロップ
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 'response_a'と'response_b'が両方'null'の行のインデックスを取得
train.drop(indexes, inplace=True)  # 取得したインデックスの行を削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット

print(f"合計 {len(indexes)} 行のNullレスポンスが削除されました")
print('トレーニングサンプルの合計: ', len(train))  # トレーニングサンプルの数を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train.head(5)
```

</div>
<div class="column-right">

# 日本語訳

```python
train.head(5)  # トレーニングデータの最初の5行を表示する
```

</div>
</details>

In [ ]:
train.head(5)  # トレーニングデータの最初の5行を表示する

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])
```

</div>
<div class="column-right">

# 日本語訳

```python
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']  
# トレーニング用のテキストを形成するために、各列のデータを結合して新しい'text'列を作成

print(train['text'][4])  # 'text'列の5行目（インデックスは4）の内容を表示
```

</div>
</details>

In [ ]:
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']  
# トレーニング用のテキストを形成するために、各列のデータを結合して新しい'text'列を作成

print(train['text'][4])  # 'text'列の5行目（インデックスは4）の内容を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Train with only take 50% train dataset
train = train[:int(len(train) * 0.5)]

train.loc[:, 'token_count'] = get_token_lengths(train['text'])

# prepare label for model
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

# Display data
display(train.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングにはデータセットの50％のみを使用
train = train[:int(len(train) * 0.5)]  # トレーニングデータの最初の50％を選択

train.loc[:, 'token_count'] = get_token_lengths(train['text'])  # 'text'列のトークン数を計算し、'token_count'列に保存

# モデル用のラベルを準備
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)  # 勝者モデルのインデックスをラベルとして設定

# データを表示
display(train.head())  # トレーニングデータの最初の数行を表示
```

</div>
</details>

In [ ]:
# トレーニングにはデータセットの50％のみを使用
train = train[:int(len(train) * 0.5)]  # トレーニングデータの最初の50％を選択

train.loc[:, 'token_count'] = get_token_lengths(train['text'])  # 'text'列のトークン数を計算し、'token_count'列に保存

# モデル用のラベルを準備
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)  # 勝者モデルのインデックスをラベルとして設定

# データを表示
display(train.head())  # トレーニングデータの最初の数行を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train.label.value_counts()
```

</div>
<div class="column-right">

# 日本語訳

```python
train.label.value_counts()  # 各ラベルの出現頻度をカウントして表示する
```

</div>
</details>

In [ ]:
train.label.value_counts()  # 各ラベルの出現頻度をカウントして表示する

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# token Count
display(train['token_count'].describe().to_frame().astype(int))
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークン数の概要を表示
display(train['token_count'].describe().to_frame().astype(int))  # 'token_count'列の統計情報を整数型のデータフレームとして表示する
```

</div>
</details>

In [ ]:
# トークン数の概要を表示
display(train['token_count'].describe().to_frame().astype(int))  # 'token_count'列の統計情報を整数型のデータフレームとして表示する

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# get length of tokens which covers 90% of data, we'll still take 1024 length!
np.percentile(train['token_count'], 90)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データの90％をカバーするトークンの長さを取得しますが、1024の長さを維持します！
np.percentile(train['token_count'], 90)  # 'token_count'列の90パーセンタイルを計算して返す
```

</div>
</details>

In [ ]:
# データの90％をカバーするトークンの長さを取得しますが、1024の長さを維持します！
np.percentile(train['token_count'], 90)  # 'token_count'列の90パーセンタイルを計算して返す

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Tokenize

</div>
<div class="column-right">

# 日本語訳

# トークナイズ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Tokenize Data
tokens = tokenizer(
    train['text'].tolist(), 
    padding='max_length', 
    max_length=CFG.MAX_LENGTH, 
    truncation=True, 
    return_tensors='np')

# Input IDs are the token IDs
INPUT_IDS = tokens['input_ids']
# Attention Masks to Ignore Padding Tokens
ATTENTION_MASKS = tokens['attention_mask']
# Label of Texts
LABELS = train[['winner_model_a','winner_model_b','winner_tie']].values

print(f'INPUT_IDS shape: {INPUT_IDS.shape}, ATTENTION_MASKS shape: {ATTENTION_MASKS.shape}')
print(f'LABELS shape: {LABELS.shape}')
```

</div>
<div class="column-right">

# 日本語訳

```python
# データをトークナイズ
tokens = tokenizer(
    train['text'].tolist(),  # トークナイズするテキストのリスト
    padding='max_length',  # 最大長にパディング
    max_length=CFG.MAX_LENGTH,  # 設定した最大長を使用
    truncation=True,  # 超過するトークンを切り捨てる
    return_tensors='np'  # NumPy形式で返す
)

# 入力IDはトークンIDに対応
INPUT_IDS = tokens['input_ids']  # トークンIDを取得
# パディングトークンを無視するためのアテンションマスク
ATTENTION_MASKS = tokens['attention_mask']  # アテンションマスクを取得
# テキストのラベル
LABELS = train[['winner_model_a','winner_model_b','winner_tie']].values  # 各テキストのラベルを取得

print(f'入力IDの形状: {INPUT_IDS.shape}, アテンションマスクの形状: {ATTENTION_MASKS.shape}')  # 入力IDとアテンションマスクの形状を出力
print(f'ラベルの形状: {LABELS.shape}')  # ラベルの形状を出力
```

</div>
</details>

In [ ]:
# データをトークナイズ
tokens = tokenizer(
    train['text'].tolist(),  # トークナイズするテキストのリスト
    padding='max_length',  # 最大長にパディング
    max_length=CFG.MAX_LENGTH,  # 設定した最大長を使用
    truncation=True,  # 超過するトークンを切り捨てる
    return_tensors='np'  # NumPy形式で返す
)

# 入力IDはトークンIDに対応
INPUT_IDS = tokens['input_ids']  # トークンIDを取得
# パディングトークンを無視するためのアテンションマスク
ATTENTION_MASKS = tokens['attention_mask']  # アテンションマスクを取得
# テキストのラベル
LABELS = train[['winner_model_a','winner_model_b','winner_tie']].values  # 各テキストのラベルを取得

print(f'入力IDの形状: {INPUT_IDS.shape}, アテンションマスクの形状: {ATTENTION_MASKS.shape}')  # 入力IDとアテンションマスクの形状を出力
print(f'ラベルの形状: {LABELS.shape}')  # ラベルの形状を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def train_dataset(batch_size):
    N_SAMPLES = LABELS.shape[0]
    IDXS = np.arange(N_SAMPLES - (N_SAMPLES % batch_size))
    while True:
        # Shuffle Indices
        np.random.shuffle(IDXS)
        # Iterate Over All Indices Once
        for idxs in IDXS.reshape(-1, batch_size):
            input_ids = torch.tensor(INPUT_IDS[idxs]).to(DEVICE)
            attention_mask = torch.tensor(ATTENTION_MASKS[idxs]).to(DEVICE)
            labels = torch.tensor(LABELS[idxs]).to(DEVICE)  # Multi-label output
            
            # Shard Over TPU Nodes if applicable (you need to define mesh appropriately)
            xs.mark_sharding(input_ids, mesh, (0, 1))
            xs.mark_sharding(attention_mask, mesh, (0, 1))
            xs.mark_sharding(labels, mesh, (0, 1))
            
            yield input_ids, attention_mask, labels

TRAIN_DATASET = train_dataset(CFG.BATCH_SIZE)
```

</div>
<div class="column-right">

# 日本語訳

```python
def train_dataset(batch_size):  # トレーニング用データセットを生成する関数
    N_SAMPLES = LABELS.shape[0]  # サンプルの数を取得
    IDXS = np.arange(N_SAMPLES - (N_SAMPLES % batch_size))  # バッチサイズで割り切れるインデックスを作成
    while True:  # 無限ループでデータを生成
        # インデックスをシャッフル
        np.random.shuffle(IDXS)  
        # すべてのインデックスを一度だけ反復
        for idxs in IDXS.reshape(-1, batch_size):  # バッチサイズのインデックスで形を変更
            input_ids = torch.tensor(INPUT_IDS[idxs]).to(DEVICE)  # 入力IDをテンソルにしてデバイスに移動
            attention_mask = torch.tensor(ATTENTION_MASKS[idxs]).to(DEVICE)  # アテンションマスクをテンソルにしてデバイスに移動
            labels = torch.tensor(LABELS[idxs]).to(DEVICE)  # ラベルをテンソルにしてデバイスに移動（マルチラベル出力）
            
            # TPUノードに対してシャーディングを行う（メッシュを適切に定義する必要があります）
            xs.mark_sharding(input_ids, mesh, (0, 1))  # 入力IDのシャーディングをマーク
            xs.mark_sharding(attention_mask, mesh, (0, 1))  # アテンションマスクのシャーディングをマーク
            xs.mark_sharding(labels, mesh, (0, 1))  # ラベルのシャーディングをマーク
            
            yield input_ids, attention_mask, labels  # 入力ID、アテンションマスク、ラベルを返す

TRAIN_DATASET = train_dataset(CFG.BATCH_SIZE)  # 定義したバッチサイズでトレーニングデータセットを作成
```

</div>
</details>

In [ ]:
def train_dataset(batch_size):  # トレーニング用データセットを生成する関数
    N_SAMPLES = LABELS.shape[0]  # サンプルの数を取得
    IDXS = np.arange(N_SAMPLES - (N_SAMPLES % batch_size))  # バッチサイズで割り切れるインデックスを作成
    while True:  # 無限ループでデータを生成
        # インデックスをシャッフル
        np.random.shuffle(IDXS)  
        # すべてのインデックスを一度だけ反復
        for idxs in IDXS.reshape(-1, batch_size):  # バッチサイズのインデックスで形を変更
            input_ids = torch.tensor(INPUT_IDS[idxs]).to(DEVICE)  # 入力IDをテンソルにしてデバイスに移動
            attention_mask = torch.tensor(ATTENTION_MASKS[idxs]).to(DEVICE)  # アテンションマスクをテンソルにしてデバイスに移動
            labels = torch.tensor(LABELS[idxs]).to(DEVICE)  # ラベルをテンソルにしてデバイスに移動（マルチラベル出力）
            
            # TPUノードに対してシャーディングを行う（メッシュを適切に定義する必要があります）
            xs.mark_sharding(input_ids, mesh, (0, 1))  # 入力IDのシャーディングをマーク
            xs.mark_sharding(attention_mask, mesh, (0, 1))  # アテンションマスクのシャーディングをマーク
            xs.mark_sharding(labels, mesh, (0, 1))  # ラベルのシャーディングをマーク
            
            yield input_ids, attention_mask, labels  # 入力ID、アテンションマスク、ラベルを返す

TRAIN_DATASET = train_dataset(CFG.BATCH_SIZE)  # 定義したバッチサイズでトレーニングデータセットを作成

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load Model

</div>
<div class="column-right">

# 日本語訳

# モデルをロード

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load model for classification with 3 target label
base_model = LlamaForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,
    num_labels=CFG.NUM_LABELS,
    torch_dtype=torch.bfloat16)

base_model.config.pretraining_tp = 1 

# Assign Padding TOKEN
base_model.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# 3つのターゲットラベルの分類用モデルをロード
base_model = LlamaForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,  # 設定したモデル名からロード
    num_labels=CFG.NUM_LABELS,  # ラベルの数を設定
    torch_dtype=torch.bfloat16)  # 使用するデータ型を指定

base_model.config.pretraining_tp = 1  # プリトレーニングのTPを設定

# パディングトークンのIDを設定
base_model.config.pad_token_id = tokenizer.pad_token_id  # トークナイザーのパディングトークンIDをモデルに設定
```

</div>
</details>

In [ ]:
# 3つのターゲットラベルの分類用モデルをロード
base_model = LlamaForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,  # 設定したモデル名からロード
    num_labels=CFG.NUM_LABELS,  # ラベルの数を設定
    torch_dtype=torch.bfloat16)  # 使用するデータ型を指定

base_model.config.pretraining_tp = 1  # プリトレーニングのTPを設定

# パディングトークンのIDを設定
base_model.config.pad_token_id = tokenizer.pad_token_id  # トークナイザーのパディングトークンIDをモデルに設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Low-Rank Adaptation [LORA]

</div>
<div class="column-right">

# 日本語訳

# 低ランク適応 [LORA]

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lora_config = LoraConfig(
    r=CFG.LORA_RANK,  # the dimension of the low-rank matrices
    lora_alpha = CFG.LORA_ALPHA, # scaling factor for LoRA activations vs pre-trained weight activations
    lora_dropout= CFG.DROPOUT, 
    bias='none',
    inference_mode=False,
    task_type=TaskType.SEQ_CLS,
    target_modules=CFG.LORA_MODULES ) # Only Use Output and Values Projection
```

</div>
<div class="column-right">

# 日本語訳

```python
lora_config = LoraConfig(  # LoRAの設定を定義
    r=CFG.LORA_RANK,  # 低ランクマトリックスの次元
    lora_alpha=CFG.LORA_ALPHA,  # LoRAアクティベーションと事前トレーニングされた重みアクティベーションのスケーリングファクター
    lora_dropout=CFG.DROPOUT,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=False,  # 推論モードの設定
    task_type=TaskType.SEQ_CLS,  # タスクの種類（シーケンス分類）
    target_modules=CFG.LORA_MODULES  # 出力と値の投影のみを使用
)
```

</div>
</details>

In [ ]:
lora_config = LoraConfig(  # LoRAの設定を定義
    r=CFG.LORA_RANK,  # 低ランクマトリックスの次元
    lora_alpha=CFG.LORA_ALPHA,  # LoRAアクティベーションと事前トレーニングされた重みアクティベーションのスケーリングファクター
    lora_dropout=CFG.DROPOUT,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=False,  # 推論モードの設定
    task_type=TaskType.SEQ_CLS,  # タスクの種類（シーケンス分類）
    target_modules=CFG.LORA_MODULES  # 出力と値の投影のみを使用
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Create LoRa Model
model = get_peft_model(base_model, lora_config)
# Trainable Parameters
model.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRAモデルを作成
model = get_peft_model(base_model, lora_config)  # 基本モデルからLoRAモデルを取得
# 訓練可能なパラメーターを表示
model.print_trainable_parameters()  # 訓練可能なパラメーターの数を出力
```

</div>
</details>

In [ ]:
# LoRAモデルを作成
model = get_peft_model(base_model, lora_config)  # 基本モデルからLoRAモデルを取得
# 訓練可能なパラメーターを表示
model.print_trainable_parameters()  # 訓練可能なパラメーターの数を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Number of TPU Nodes
num_devices = xr.global_runtime_device_count()
mesh_shape = (1, num_devices, 1)
device_ids = np.array(range(num_devices))
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))
# distribute model
partition_module(model, mesh)

print(f'num_devices: {num_devices}')
```

</div>
<div class="column-right">

# 日本語訳

```python
# TPUノードの数を取得
num_devices = xr.global_runtime_device_count()  # 使用可能なTPUデバイスの数を取得
mesh_shape = (1, num_devices, 1)  # メッシュの形状を定義
device_ids = np.array(range(num_devices))  # デバイスIDの配列を作成
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))  # メッシュを作成（データ並列、フルモデルの分散、モデル並列）

# モデルを分散させる
partition_module(model, mesh)  # モジュールをメッシュに基づいてパーティショニング

print(f'デバイスの数: {num_devices}')  # 使用するデバイスの数を出力
```

</div>
</details>

In [ ]:
# TPUノードの数を取得
num_devices = xr.global_runtime_device_count()  # 使用可能なTPUデバイスの数を取得
mesh_shape = (1, num_devices, 1)  # メッシュの形状を定義
device_ids = np.array(range(num_devices))  # デバイスIDの配列を作成
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))  # メッシュを作成（データ並列、フルモデルの分散、モデル並列）

# モデルを分散させる
partition_module(model, mesh)  # モジュールをメッシュに基づいてパーティショニング

print(f'デバイスの数: {num_devices}')  # 使用するデバイスの数を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Verfy The Trainable Layers
MODEL_LAYERS_ROWS = []
TRAINABLE_PARAMS = []
N_TRAINABLE_PARAMS = 0

for name, param in model.named_parameters():
    # Layer Parameter Count
    n_parameters = int(torch.prod(torch.tensor(param.shape)))
    # Only Trainable Layers
    if param.requires_grad:
        # Add Layer Information
        MODEL_LAYERS_ROWS.append({
            'param': n_parameters,
            'name': name,
            'dtype': param.data.dtype,
        })
        # Append Trainable Parameter
        TRAINABLE_PARAMS.append({ 'params': param })
        # Add Number Of Trainable Parameters"
        N_TRAINABLE_PARAMS += n_parameters
        
display(pd.DataFrame(MODEL_LAYERS_ROWS))

print(f"""
===============================
N_TRAINABLE_PARAMS: {N_TRAINABLE_PARAMS:,}
N_TRAINABLE_LAYERS: {len(TRAINABLE_PARAMS)}
===============================
""")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習可能なレイヤーを確認
MODEL_LAYERS_ROWS = []  # モデルのレイヤー情報を格納するリスト
TRAINABLE_PARAMS = []  # 学習可能なパラメータを格納するリスト
N_TRAINABLE_PARAMS = 0  # 学習可能なパラメータの合計数

for name, param in model.named_parameters():  # モデルの各パラメータについてループ
    # レイヤーのパラメータ数を計算
    n_parameters = int(torch.prod(torch.tensor(param.shape)))  
    # 学習可能なレイヤーのみ確認
    if param.requires_grad:  
        # レイヤー情報を追加
        MODEL_LAYERS_ROWS.append({
            'param': n_parameters,  # パラメータ数
            'name': name,  # レイヤー名
            'dtype': param.data.dtype,  # データ型
        })
        # 学習可能なパラメータを追加
        TRAINABLE_PARAMS.append({'params': param})  
        # 学習可能なパラメータの数を増加
        N_TRAINABLE_PARAMS += n_parameters
        
display(pd.DataFrame(MODEL_LAYERS_ROWS))  # レイヤー情報のデータフレームを表示

print(f"""
===============================
学習可能なパラメータの数: {N_TRAINABLE_PARAMS:,}
学習可能なレイヤーの数: {len(TRAINABLE_PARAMS)}
===============================
""")  # 学習可能なパラメータの総数とレイヤーの数を出力
```

</div>
</details>

In [ ]:
# 学習可能なレイヤーを確認
MODEL_LAYERS_ROWS = []  # モデルのレイヤー情報を格納するリスト
TRAINABLE_PARAMS = []  # 学習可能なパラメータを格納するリスト
N_TRAINABLE_PARAMS = 0  # 学習可能なパラメータの合計数

for name, param in model.named_parameters():  # モデルの各パラメータについてループ
    # レイヤーのパラメータ数を計算
    n_parameters = int(torch.prod(torch.tensor(param.shape)))  
    # 学習可能なレイヤーのみ確認
    if param.requires_grad:  
        # レイヤー情報を追加
        MODEL_LAYERS_ROWS.append({
            'param': n_parameters,  # パラメータ数
            'name': name,  # レイヤー名
            'dtype': param.data.dtype,  # データ型
        })
        # 学習可能なパラメータを追加
        TRAINABLE_PARAMS.append({'params': param})  
        # 学習可能なパラメータの数を増加
        N_TRAINABLE_PARAMS += n_parameters
        
display(pd.DataFrame(MODEL_LAYERS_ROWS))  # レイヤー情報のデータフレームを表示

print(f"""
===============================
学習可能なパラメータの数: {N_TRAINABLE_PARAMS:,}
学習可能なレイヤーの数: {len(TRAINABLE_PARAMS)}
===============================
""")  # 学習可能なパラメータの総数とレイヤーの数を出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Training

</div>
<div class="column-right">

# 日本語訳

# トレーニング

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# LR & Optimizer
N_SAMPLES = len(train)
STEPS_PER_EPOCH = N_SAMPLES // CFG.BATCH_SIZE

OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=CFG.LR_MAX)

# Cosine Learning Rate With Warmup
lr_scheduler = transformers.get_cosine_schedule_with_warmup(
    optimizer=OPTIMIZER,
    num_warmup_steps=CFG.NUM_WARMUP_STEPS,
    num_training_steps=STEPS_PER_EPOCH * CFG.NUM_EPOCHS)

print(f'BATCH_SIZE: {CFG.BATCH_SIZE}, N_SAMPLES: {N_SAMPLES}, STEPS_PER_EPOCH: {STEPS_PER_EPOCH}')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習率 & オプティマイザーの設定
N_SAMPLES = len(train)  # トレーニングサンプルの数を取得
STEPS_PER_EPOCH = N_SAMPLES // CFG.BATCH_SIZE  # エポックごとのステップ数を計算

OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=CFG.LR_MAX)  # AdamWオプティマイザーを設定

# コサイン学習率スケジュール（ウォームアップ付き）
lr_scheduler = transformers.get_cosine_schedule_with_warmup(
    optimizer=OPTIMIZER,  # 使用するオプティマイザー
    num_warmup_steps=CFG.NUM_WARMUP_STEPS,  # ウォームアップのステップ数
    num_training_steps=STEPS_PER_EPOCH * CFG.NUM_EPOCHS)  # 総トレーニングステップ数

print(f'バッチサイズ: {CFG.BATCH_SIZE}, サンプル数: {N_SAMPLES}, エポックごとのステップ数: {STEPS_PER_EPOCH}')  # バッチサイズ、サンプル数、エポックごとのステップ数を出力
```

</div>
</details>

In [ ]:
# 学習率 & オプティマイザーの設定
N_SAMPLES = len(train)  # トレーニングサンプルの数を取得
STEPS_PER_EPOCH = N_SAMPLES // CFG.BATCH_SIZE  # エポックごとのステップ数を計算

OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=CFG.LR_MAX)  # AdamWオプティマイザーを設定

# コサイン学習率スケジュール（ウォームアップ付き）
lr_scheduler = transformers.get_cosine_schedule_with_warmup(
    optimizer=OPTIMIZER,  # 使用するオプティマイザー
    num_warmup_steps=CFG.NUM_WARMUP_STEPS,  # ウォームアップのステップ数
    num_training_steps=STEPS_PER_EPOCH * CFG.NUM_EPOCHS)  # 総トレーニングステップ数

print(f'バッチサイズ: {CFG.BATCH_SIZE}, サンプル数: {N_SAMPLES}, エポックごとのステップ数: {STEPS_PER_EPOCH}')  # バッチサイズ、サンプル数、エポックごとのステップ数を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Set the data type for the optimizer's state (e.g., momentum buffers)
for state in OPTIMIZER.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor) and state[k].dtype is not torch.float32:
            state[v] = v.to(dtype=torch.float32)
```

</div>
<div class="column-right">

# 日本語訳

```python
# オプティマイザーの状態（例：モーメンタムバッファ）に対するデータ型を設定
for state in OPTIMIZER.state.values():  # オプティマイザーのすべての状態についてループ
    for k, v in state.items():  # 各状態のキーと値についてループ
        if isinstance(v, torch.Tensor) and state[k].dtype is not torch.float32:  # テンソルであり、データ型がfloat32でない場合
            state[v] = v.to(dtype=torch.float32)  # テンソルのデータ型をfloat32に変換
```

</div>
</details>

In [ ]:
# オプティマイザーの状態（例：モーメンタムバッファ）に対するデータ型を設定
for state in OPTIMIZER.state.values():  # オプティマイザーのすべての状態についてループ
    for k, v in state.items():  # 各状態のキーと値についてループ
        if isinstance(v, torch.Tensor) and state[k].dtype is not torch.float32:  # テンソルであり、データ型がfloat32でない場合
            state[v] = v.to(dtype=torch.float32)  # テンソルのデータ型をfloat32に変換

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
input_ids, attention_mask, labels = next(TRAIN_DATASET)

print(f'input_ids shape: {input_ids.shape}, dtype: {input_ids.dtype}')
print(f'attention_mask shape: {attention_mask.shape}, dtype: {attention_mask.dtype}')
print(f'labels shape: {labels.shape}, dtype: {labels.dtype}')
```

</div>
<div class="column-right">

# 日本語訳

```python
input_ids, attention_mask, labels = next(TRAIN_DATASET)  # トレーニングデータセットから次のバッチを取得

print(f'入力IDの形状: {input_ids.shape}, データ型: {input_ids.dtype}')  # 入力IDの形状とデータ型を出力
print(f'アテンションマスクの形状: {attention_mask.shape}, データ型: {attention_mask.dtype}')  # アテンションマスクの形状とデータ型を出力
print(f'ラベルの形状: {labels.shape}, データ型: {labels.dtype}')  # ラベルの形状とデータ型を出力
```

</div>
</details>

In [ ]:
input_ids, attention_mask, labels = next(TRAIN_DATASET)  # トレーニングデータセットから次のバッチを取得

print(f'入力IDの形状: {input_ids.shape}, データ型: {input_ids.dtype}')  # 入力IDの形状とデータ型を出力
print(f'アテンションマスクの形状: {attention_mask.shape}, データ型: {attention_mask.dtype}')  # アテンションマスクの形状とデータ型を出力
print(f'ラベルの形状: {labels.shape}, データ型: {labels.dtype}')  # ラベルの形状とデータ型を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time
# Dummy Prediction
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
print(f'logits: {outputs.logits}, dtype: {outputs.logits.dtype}')
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time
# ダミー予測
with torch.no_grad():  # 勾配計算を無効にする
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # モデルの出力を取得
    
print(f'ロジット: {outputs.logits}, データ型: {outputs.logits.dtype}')  # ロジットの値とデータ型を出力
```

</div>
</details>

In [ ]:
%%time
# ダミー予測
with torch.no_grad():  # 勾配計算を無効にする
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # モデルの出力を取得
    
print(f'ロジット: {outputs.logits}, データ型: {outputs.logits.dtype}')  # ロジットの値とデータ型を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Put Model In Train Mode
model.train()

# Loss Function, Cross Entropy
LOSS_FN = torch.nn.CrossEntropyLoss().to(dtype=torch.float32)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルをトレーニングモードに設定
model.train()  # モデルをトレーニングモードに切り替え

# 損失関数、クロスエントロピー
LOSS_FN = torch.nn.CrossEntropyLoss().to(dtype=torch.float32)  # クロスエントロピー損失を定義し、データ型をfloat32に設定
```

</div>
</details>

In [ ]:
# モデルをトレーニングモードに設定
model.train()  # モデルをトレーニングモードに切り替え

# 損失関数、クロスエントロピー
LOSS_FN = torch.nn.CrossEntropyLoss().to(dtype=torch.float32)  # クロスエントロピー損失を定義し、データ型をfloat32に設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time()
warnings.filterwarnings("error")
METRICS = {
    'loss': [],
    'accuracy': {'y_true': [], 'y_pred': [] }}

for epoch in tqdm(range(CFG.NUM_EPOCHS)):
    ste = time()
    for step in range(STEPS_PER_EPOCH):
        # Zero Out Gradients
        OPTIMIZER.zero_grad()
        
        # Get Batch
        input_ids, attention_mask, labels = next(TRAIN_DATASET)
        
        # Forward Pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
       
        # Logits Float32
        logits = outputs.logits.to(dtype=torch.float32)
        
        # Backward Pass
        loss = LOSS_FN(logits, labels.to(dtype=torch.float32))
        loss.backward()
        
        # optimizer step
        OPTIMIZER.step()
        xm.mark_step()
        
        # Update Learning Rate Scheduler
        lr_scheduler.step()
        
        # Update Metrics And Progress Bar
        METRICS['loss'].append(float(loss))
        METRICS['accuracy']['y_true'] += labels.squeeze().tolist()
        METRICS['accuracy']['y_pred'] += torch.argmax(F.softmax(logits, dim=-1), dim=1).cpu().tolist()
        
        if (step + 1) % 200 == 0:  
            metrics = 'µ_loss: {:.3f}'.format(np.mean(METRICS['loss']))
            metrics += ', step_loss: {:.3f}'.format(METRICS['loss'][-1])
            metrics += ', µ_auc: {:.3f}'.format(accuracy_score(torch.argmax(torch.tensor(METRICS['accuracy']['y_true']), axis=-1), \
                                                               METRICS['accuracy']['y_pred']))
            lr = OPTIMIZER.param_groups[0]['lr']
            print(f'{epoch+1:02}/{CFG.NUM_EPOCHS:02} | {step+1:04}/{STEPS_PER_EPOCH} lr: {lr:.2E}, {metrics}', end='')
            print(f'\nSteps per epoch: {step+1} complete | Time elapsed: {time()- st}')
    
    print(f'\nEpoch {epoch+1} Completed | Total time for epoch: {time() - ste} ' )

    # If stopped, and to continue training in future on tpu we save model and optimizer
    xm.save({k: v.cpu() for k, v in model.named_parameters() if v.requires_grad}, f'model_llama_3_cp_{epoch+1}_v1.pth')
    xm.save(OPTIMIZER.state_dict(), f'optimizer_llama_3_cp_{epoch+1}_v1.pth')    
    
    print(f'Model saved at epoch {epoch+1}| Elapsed time: {time() - st} ')
```

</div>
<div class="column-right">

# 日本語訳

```python
st = time()  # 開始時刻を記録
warnings.filterwarnings("error")  # 警告をエラーとして扱う
METRICS = {
    'loss': [],  # 損失を記録するリスト
    'accuracy': {'y_true': [], 'y_pred': [] }  # 正解ラベルと予測ラベルを記録する辞書
}

for epoch in tqdm(range(CFG.NUM_EPOCHS)):  # エポック数だけループ
    ste = time()  # 各エポックの開始時刻を記録
    for step in range(STEPS_PER_EPOCH):  # 各エポック内のステップ数だけループ
        # 勾配をゼロに設定
        OPTIMIZER.zero_grad()
        
        # バッチを取得
        input_ids, attention_mask, labels = next(TRAIN_DATASET)
        
        # フォワードパス
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
       
        # ロジットをfloat32に変換
        logits = outputs.logits.to(dtype=torch.float32)
        
        # バックワードパス
        loss = LOSS_FN(logits, labels.to(dtype=torch.float32))  # 損失を計算
        loss.backward()  # バックワードパスの実行
        
        # オプティマイザーのステップ
        OPTIMIZER.step()  # オプティマイザーを更新
        xm.mark_step()  # TPUデバイスのステップをマーク
        
        # 学習率スケジューラーを更新
        lr_scheduler.step()
        
        # メトリクスと進行状況バーを更新
        METRICS['loss'].append(float(loss))  # 損失を記録
        METRICS['accuracy']['y_true'] += labels.squeeze().tolist()  # 正解ラベルを記録
        METRICS['accuracy']['y_pred'] += torch.argmax(F.softmax(logits, dim=-1), dim=1).cpu().tolist()  # 予測ラベルを記録
        
        if (step + 1) % 200 == 0:  # 200ステップごとに進捗を表示
            metrics = 'µ_loss: {:.3f}'.format(np.mean(METRICS['loss']))  # 平均損失を計算
            metrics += ', step_loss: {:.3f}'.format(METRICS['loss'][-1])  # 最後の損失を表示
            metrics += ', µ_auc: {:.3f}'.format(accuracy_score(torch.argmax(torch.tensor(METRICS['accuracy']['y_true']), axis=-1), \
                                                               METRICS['accuracy']['y_pred']))  # 正確度を計算
            lr = OPTIMIZER.param_groups[0]['lr']  # 現在の学習率を取得
            print(f'{epoch+1:02}/{CFG.NUM_EPOCHS:02} | {step+1:04}/{STEPS_PER_EPOCH} lr: {lr:.2E}, {metrics}', end='')
            print(f'\nエポック内のステップ: {step+1} 完了 | 経過時間: {time()- st}')  # 経過時間を表示
    
    print(f'\nエポック {epoch+1} 完了 | エポック全体の経過時間: {time() - ste} ' )

    # 停止した場合、将来再トレーニングを行うためにモデルとオプティマイザーを保存
    xm.save({k: v.cpu() for k, v in model.named_parameters() if v.requires_grad}, f'model_llama_3_cp_{epoch+1}_v1.pth')  # モデルを保存
    xm.save(OPTIMIZER.state_dict(), f'optimizer_llama_3_cp_{epoch+1}_v1.pth')  # オプティマイザーを保存    
    
    print(f'エポック {epoch+1} でモデルが保存されました | 経過時間: {time() - st} ')
```

</div>
</details>

In [ ]:
st = time()  # 開始時刻を記録
warnings.filterwarnings("error")  # 警告をエラーとして扱う
METRICS = {
    'loss': [],  # 損失を記録するリスト
    'accuracy': {'y_true': [], 'y_pred': [] }  # 正解ラベルと予測ラベルを記録する辞書
}

for epoch in tqdm(range(CFG.NUM_EPOCHS)):  # エポック数だけループ
    ste = time()  # 各エポックの開始時刻を記録
    for step in range(STEPS_PER_EPOCH):  # 各エポック内のステップ数だけループ
        # 勾配をゼロに設定
        OPTIMIZER.zero_grad()
        
        # バッチを取得
        input_ids, attention_mask, labels = next(TRAIN_DATASET)
        
        # フォワードパス
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
       
        # ロジットをfloat32に変換
        logits = outputs.logits.to(dtype=torch.float32)
        
        # バックワードパス
        loss = LOSS_FN(logits, labels.to(dtype=torch.float32))  # 損失を計算
        loss.backward()  # バックワードパスの実行
        
        # オプティマイザーのステップ
        OPTIMIZER.step()  # オプティマイザーを更新
        xm.mark_step()  # TPUデバイスのステップをマーク
        
        # 学習率スケジューラーを更新
        lr_scheduler.step()
        
        # メトリクスと進行状況バーを更新
        METRICS['loss'].append(float(loss))  # 損失を記録
        METRICS['accuracy']['y_true'] += labels.squeeze().tolist()  # 正解ラベルを記録
        METRICS['accuracy']['y_pred'] += torch.argmax(F.softmax(logits, dim=-1), dim=1).cpu().tolist()  # 予測ラベルを記録
        
        if (step + 1) % 200 == 0:  # 200ステップごとに進捗を表示
            metrics = 'µ_loss: {:.3f}'.format(np.mean(METRICS['loss']))  # 平均損失を計算
            metrics += ', step_loss: {:.3f}'.format(METRICS['loss'][-1])  # 最後の損失を表示
            metrics += ', µ_auc: {:.3f}'.format(accuracy_score(torch.argmax(torch.tensor(METRICS['accuracy']['y_true']), axis=-1), \
                                                               METRICS['accuracy']['y_pred']))  # 正確度を計算
            lr = OPTIMIZER.param_groups[0]['lr']  # 現在の学習率を取得
            print(f'{epoch+1:02}/{CFG.NUM_EPOCHS:02} | {step+1:04}/{STEPS_PER_EPOCH} lr: {lr:.2E}, {metrics}', end='')
            print(f'\nエポック内のステップ: {step+1} 完了 | 経過時間: {time()- st}')  # 経過時間を表示
    
    print(f'\nエポック {epoch+1} 完了 | エポック全体の経過時間: {time() - ste} ' )

    # 停止した場合、将来再トレーニングを行うためにモデルとオプティマイザーを保存
    xm.save({k: v.cpu() for k, v in model.named_parameters() if v.requires_grad}, f'model_llama_3_cp_{epoch+1}_v1.pth')  # モデルを保存
    xm.save(OPTIMIZER.state_dict(), f'optimizer_llama_3_cp_{epoch+1}_v1.pth')  # オプティマイザーを保存    
    
    print(f'エポック {epoch+1} でモデルが保存されました | 経過時間: {time() - st} ')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(15, 6))
plt.plot(METRICS['loss'])    
plt.xlabel('Step per epoch')
plt.ylabel('Loss')
plt.title('Loss Plot step per epoch')    
plt.show()
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(15, 6))  # グラフのサイズを設定
plt.plot(METRICS['loss'])  # 損失の変化をプロット
plt.xlabel('エポック内のステップ')  # x軸のラベル
plt.ylabel('損失')  # y軸のラベル
plt.title('エポック内のステップに対する損失プロット')  # グラフのタイトル
plt.show()  # グラフを表示
```

</div>
</details>

In [ ]:
plt.figure(figsize=(15, 6))  # グラフのサイズを設定
plt.plot(METRICS['loss'])  # 損失の変化をプロット
plt.xlabel('エポック内のステップ')  # x軸のラベル
plt.ylabel('損失')  # y軸のラベル
plt.title('エポック内のステップに対する損失プロット')  # グラフのタイトル
plt.show()  # グラフを表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Save Model


</div>
<div class="column-right">

# 日本語訳

# モデルを保存

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = model.cpu()
torch.save(dict([(k,v) for k, v in model.named_parameters() if v.requires_grad]), 'llama_3_finetuned_model.pth')
```

</div>
<div class="column-right">

# 日本語訳

```python
model = model.cpu()  # モデルをCPUに移動
torch.save(dict([(k, v) for k, v in model.named_parameters() if v.requires_grad]), 'llama_3_finetuned_model.pth')  
# 学習可能なパラメータのみを保存し、'llama_3_finetuned_model.pth'というファイル名で保存
```

</div>
</details>

In [ ]:
model = model.cpu()  # モデルをCPUに移動
torch.save(dict([(k, v) for k, v in model.named_parameters() if v.requires_grad]), 'llama_3_finetuned_model.pth')  
# 学習可能なパラメータのみを保存し、'llama_3_finetuned_model.pth'というファイル名で保存

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Conclusion 

There is still alot of room to speed up and optimize training! Try out more data, different batch size, lr... All the best!

</div>
<div class="column-right">

# 日本語訳

# 結論

トレーニングを加速させ、最適化する余地がまだたくさんあります！もっと多くのデータ、異なるバッチサイズ、学習率などを試してみてください。成功を祈っています！

</div>